# Convert a spreadsheet to a website.

A couple of rules:

Name your pages like ```App layout``` or ```Shop layout```, with an optional ```Shop data``` which will automatically load as a data table to be used in the template like ```{{#shop-data}}{{description}}{{/shop-data}}```. This would repeat for every row in the data table printing out all of the description columns. Data tables should list the column names as the first row. Hiding sheets doesn't matter to the converter but makes it easier to manage lots of pages or see only the one's you are working on.

Use HTML or Mustache or Markdown in your sheets. Rows are automatically converted to top level ```<div>``` elements. The class ```col-2``` is added to count the number of columns. Cells are automatically converted to ```<div>``` as children on the row ```<div>```. The class ```cell-0``` is automatically added as the index of the cell, not including any variables in between.

Beginning a cell with ":" comma creates variables and the cell right of a variable is used at the value. Variables do not affect column count or HTML output. Variables are combined with subpages, that is, any variable set on one page can be used page another page loaded by a URL, ```{{> section}}```, or ```::render```.

Special variables are:

```:logo```, which shows up in the favicon.

```:title``` is used as page title.

```::render``` forces another template to render whether or not there is a link to it, this is a list and can be used as many times as you want.

```{{> section}}``` inserts a {{#section}}{{/section}} mustache template, combines section from mustach and render from this framework. Pages that are not linked to, rendered, or sectioned, won't be scanned at all. Sections should be refered to in lowercase with spaces converted to dashes and without the word ```layout``` attached, i.e. ```{{> shop-menu}}```.

URLs such as ```/link``` inherently forces the spreadsheet page named Link layout, or Link data to render. Links to other pages are automatically detected. Names of pages are automatically converted to lowercase, with dashes instead of spaces.

[Markdown description at markdownguide.org](https://www.markdownguide.org/basic-syntax)

[Mustache guide from NodeJs](https://github.com/janl/mustache.js)


TODO:

Use ```::variable``` to create a list such as ```::stylesheet``` or ```:scripts```.

Inlining all styles and scripts and images as data-uri for faster load times.

Automatically detecting which features to add such as ```Buy Now``` buttons and contact us chats.


google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');

function getTemplateProperties(key, properties, templates) {
    var returnObj = {};
    var promise;
    if(typeof properties[key + '-data'] != 'undefined') {
        promise = Promise.resolve(properties[key + '-data']);
    } else if(templates[key].data) {
        promise = getDataSheet(templates[key].data);
    } else {
        promise = Promise.resolve();
    }
    
    return promise
        .then(data => {
            properties[key + '-data'] = data;
            returnObj[key + '-data'] = data;
            var templatePromise;
            if(typeof properties[key] != 'undefined') {
                return Promise.resolve(properties[key]);
            } else {
                return getSections(key, properties, templates)
                    .then(rows => renderRows(key, rows, properties, templates));
            }
        })
        .then(template => {
            properties[key] = template;
            returnObj[key] = template;
            return returnObj;
        });
}

function filteredData(value, key, properties) {
    return function(val, render) {
        var match = value.split('/')[1];
        var l = properties[match]
        var matchKey = key + '-' + l + '-data';
        properties[matchKey] = properties[key + '-data']
            .filter(row => row[key] == l || row[key] == l.split('/')[2]);
        console.log(`Rendering filtered: ${key} ${match} ${l} ${properties[matchKey].length}`);
        return render(`{{#${matchKey}}}${val}{{/${matchKey}}}`);
    }
}

// must do this up front so we can process all data
function getSections(layoutTitle, properties, templates) {
    var worksheet = templates[layoutTitle].template;
    if(!worksheet) {
        return Promise.resolve();
    }
    // TODO: error check this?
    //if(typeof properties[layoutTitle] != 'undefined') {
    //    return Promise.resolve(properties[layoutTitle]);
    //}

    var rows;
    return getRows(worksheet)
        .then(rs => {
            rows = rs;
            return rows.reduce((promise, row, i) => {
                return row.reduce((promise, c, j) => {
                    return importer.regexToArray(/\{\{>\s*(.*?)\}\}/ig, c, 1).reduce((promise, value) => {
                        var segments = value.split('/');
                        var key = segments[0];

                        if(typeof templates[key] != 'undefined') {
                            console.log(`Reading partial: ${key}`);
                            return promise
                                .then(() => getTemplateProperties(key, properties, templates))
                                .then(() => {
                                    // add a special partial for the filtered data
                                    if(segments.length > 1) {
                                        properties[value] = properties[key];
                                        properties[value + '-data'] = filteredData.bind(
                                            null, value, key, properties);
                                    }
                                })
                        } else {
                            console.log(`Section ${key} not found!`);
                            return promise;
                        }
                    }, promise)
                }, promise);
            }, Promise.resolve());
        })
        .then(() => rows)
}

module.exports = getTemplateProperties;

google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var importer = require('../Core');
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

function safeName(name) {
    return name.replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function escape(s) {
    return s.replace(/[-\/\\^$*+?.()|[\]{}]/g, '\\$&');
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    
    return dataKeys
        .filter(k => c.match(new RegExp('\\{\\{\\s*[>#\\/]?\\s*' + escape(k) + '\\s*\\}\\}', 'ig')))
}

function renderRows(layoutTitle, rows, properties, templates) {
    if(!rows) {
        rows = [];
    }
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            var dataClasses = getDataClasses(c, properties[layoutTitle + '-data'])
                .map(k => 'val-' + safeName(k))
                .join(' ')
            var sectionClasses = getDataClasses(c, [properties, templates])
                .map(k => 'section-' + safeName(k))
                .join(' ')
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                var value = row[j + 1];
                if (c.substr(0, 2) === '::' || c === ':render') {
                    if(c === ':render') c = '::render'; // just to fix using it below
                    if(typeof properties[c.substr(2)] == 'undefined') {
                        properties[c.substr(2)] = [];
                    }
                    properties[c.substr(2)][properties[c.substr(2)].length] = value;
                } else {
                    properties[c.substr(1)] = value;
                }
            // render markdown content if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1] && row[j - 1].substr(0, 1) !== ':') {
                var mdHtml = md.render(c).replace(/\{\{\s*&gt;\s*/ig, '{{> ');
                // if all markdown did was insert a paragraph and line break, use value instead
                if(mdHtml.replace(/<\/?p>|[\s\n\r]+/ig, '').trim() == c.replace(/<\/?p>|[\s\n\r]+/ig, '').trim()) {
                    mdHtml = c;
                }

                arr[arr.length] = `
<div class="cell-${arr.length} ${dataClasses} ${sectionClasses}">
${mdHtml}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            arr[arr.length] = `
<div class="row-${arr.length} ${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;

output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function safeName(val, render) {
    return render(val).replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function wrapTemplate(layoutTitle, html, properties) {
    properties['safeName'] = () => safeName;
    // TODO: turn this in to an array/section?
    var stylesheet = '', script = '', banner = '', domain = '';
    if(typeof properties['stylesheet'] != 'undefined') {
        if(typeof properties['stylesheet'] === 'string') {
            properties['stylesheet'] = [properties['stylesheet']];
        }
        stylesheet = '{{#stylesheet}}<link rel="stylesheet" media="screen" href="{{.}}">{{/stylesheet}}';
    }
    if(typeof properties['script'] != 'undefined') {
        if(typeof properties['script'] === 'string') {
            properties['script'] = [properties['script']];
        }
        script = '{{#script}}<script src="{{.}}"></script>{{/script}}';
    }
    if(typeof properties['banner'] != 'undefined') {
        banner = 'background-image: url({{{banner}}});';
    }
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '{{domain}}' : ('https://{{domain}}');
    }
    
    // automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
${stylesheet}
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${layoutTitle}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
${banner}
}
</style>
</head>
<body class="${layoutTitle.replace(/\//ig, ' ')} {{class}}">
${html}
${script}
</body>
</html>`;
    Mustache.parse(pageHtml);
    // use properties for view and for partials
    return Mustache.render(pageHtml, properties, properties);
}

module.exports = wrapTemplate;


find known routes to sheets?


In [ ]:
var importer = require('../Core');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');

function overwriteProperties(l, properties, templates) {
    var urlParts = l.split('/');
    var key = urlParts[1];
    var fileName = key;
    var newProps = Object.assign({}, properties);
    return getTemplateProperties(key, newProps, templates)
        .then(() => {
            // if rendering a page has url parts
            if(urlParts.length > 2) {
                // use the third argument to find the correct row in the data set
                var rowData = newProps[key + '-data']
                    .filter(row => row['link'] == l || row['url'] == l)[0];
                // handle multiple results for use with categories
                if(typeof rowData === 'undefined') {
                    rowData = {};
                    rowData[key + '-data'] = newProps[key + '-data']
                        .filter(row => row[key] == l || row[key] == l.split('/')[2]);
                    // assign the matching value to the template property
                    rowData[key] = rowData[key + '-data'][0][key];
                }
                fileName = l.substr(1);
                return wrapTemplate(fileName, newProps[key], Object.assign(newProps, rowData))
            }
            return wrapTemplate(fileName, newProps[key], newProps)
        })
        .then(page => {
            var returnObj = {};
            returnObj[fileName] = page;
            return returnObj;
        })
}

function collectRoutes(routes, properties, templates, rendered) {
    var local = routes.concat(properties['render'] || [])
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':'))
        .filter(l => !rendered.includes(l) && typeof templates[l.split('/')[1]] != 'undefined')

    local.forEach(l => rendered[rendered.length] = l)
    
    return importer.runAllPromises(local
        // promise in series so there is no data collisions
        .map(l => resolve => overwriteProperties(l, properties, templates)
            .then(page => resolve(page))))
        .then(results => results.reduce((obj, r) => Object.assign(obj, r), {}))
}

module.exports = collectRoutes;



collect external content and resources?


In [ ]:
var {Readable} = require('stream');
var {JSDOM, XPathResult} = require('jsdom');
var importer = require('../Core');
var getArrayXPath = importer.import('get xpath array');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

function collectExternalResources(page, rendered, routes) {
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    // TODO: add IDs to h1, h2, h3, etc elements that match their text contents
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    // TODO: convert images and add timestamps, add svg
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    // TODO: scan for urls and inline
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    // TODO: add timestamps and inline
    var scriptsObjs = getArrayXPath('//script[@src]', dom.window.document.body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    // TODO: add CSS imports
    var backgrounds = regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var searches = imgs.concat(styles).concat(backgrounds).concat(scripts)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && rendered.indexOf(cur) === -1);
    
    links.forEach(s => routes[routes.length] = s)
    searches.forEach(s => rendered[rendered.length] = s)
    
    // TODO: copy resource images to output directory
    var newPage = dom.window.document.documentElement.outerHTML
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

module.exports = collectExternalResources;


collect google sheets resources?


In [ ]:
var importer = require('../Core');
var streamToGoogle = importer.import('upload files google cloud');
var copyFileBucket = importer.import('copy file bucket storage');
var collectExternalResources = importer.import('collect external content and resources');
var collectRoutes = importer.import('find known routes to sheets');

var timestamp = (new Date()).getTime();

// detect links and write out every part of the site
function collectTemplateResources(layoutTitle, page, properties, templates, bucketName, rendered, sTG) {
    console.log(`Rendering ${layoutTitle}`);
    
    if(!rendered) {
        rendered = [];
    }
    
    // if it is the first page in the template, rename it to index.html
    var template = layoutTitle.split('/')[0];
    if(templates[template] && templates[template].template
       && templates[template].template.properties.index === 0) {
        console.log(`Using ${layoutTitle} as index.html`);
        layoutTitle = 'index';
    }

    // TODO: add timestamps to generated content
    var routes = [];
    return collectExternalResources(page, rendered, routes)
        .then(stream => (sTG ? sTG : streamToGoogle)(layoutTitle + '.html', bucketName, stream, {
            contentType: 'text/html; charset=utf-8'
        }))
        .then(() => !sTG ? copyFileBucket(bucketName, layoutTitle + '.html') : Promise.resolve(false))
        .then(() => collectRoutes(routes, properties, templates, rendered))
        .then(pages => importer.runAllPromises(Object.keys(pages).map(fileName => resolve => 
            collectTemplateResources(fileName, pages[fileName], properties, templates, bucketName, rendered, sTG)
                .then(() => resolve()))))
        .then(() => rendered)
}

module.exports = collectTemplateResources;

test google sheets resources?


In [ ]:
var assert = require('assert');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var findSimilarFile = importer.import('find similar file');

var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));

// locally based utility for editing styles
function streamToOutput(fileUrl, bucketName, stream) {    
     return new Promise((resolve, reject) => {
        if(typeof stream == 'object') {
            var outputPath = path.join(path.resolve('./.output/'), fileUrl.replace(/\?.*/ig, ''));
            if(!fs.existsSync(path.dirname(outputPath))) {
                fs.mkdirSync(path.dirname(outputPath));
            }
            var writeStream = fs.createWriteStream(outputPath);
            stream.pipe(writeStream)
            .on("error", (err) => {
                reject(err);
            })
            .on('finish', () => {
                resolve(outputPath);
            });
        } else {
            var outputPath = path.join(path.resolve('./.output/'), path.basename(fileUrl.replace(/\?.*/ig, '')));
            var writeStream = fs.createWriteStream(outputPath);
            (fileUrl.includes('https://') ? https : http).get(fileUrl, response => {
                response.pipe(writeStream)
                .on("error", (err) => {
                    reject(err);
                })
                .on('finish', () => {
                    resolve(outputPath);
                });
            }).catch(e => console.log(e));
        }
     });
}

function outputTest(page, bucketName, key, templates, properties) {
    return collectTemplateResources(key, page, properties, templates, bucketName, false, streamToOutput)
        .then(resources => {
            console.log(resources);
            return Promise.all(resources.filter(r => !r.includes(':') && r.includes('.'))
                .map(r => {
                    // TODO: search notebooks for code so library can be saved to git
                    return findSimilarFile(path.basename(r), DOWNLOAD_DIR)
                        .then(f => {
                            if(f) {
                                return streamToOutput(
                                    f,
                                    bucketName,
                                    fs.createReadStream(path.join(DOWNLOAD_DIR, f)));
                            } else {
                                return Promise.resolve(false);
                            }
                        });
                }));
        });
}

function importTest(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var templates, key, properties = {};
    
    return getTemplates(docId)
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => {
            assert(page.length > 0, 'should have a page');
            return outputTest(page, domain, key, templates, properties);
        })
}

if(typeof describe !== 'undefined') {
    describe('output template resources', () => {

        it('should process at least one template', () => {
            var docsId = '1QeZ3WduNFmjtNf_Q3Xe5zAh8dzP_nn7nr-AQU0_hXg8';
            var bucketName = 'sheet-to-web.com';
            return importTest(docsId, bucketName);
        }).timeout(60000)
    })
    
}

module.exports = importTest;


get sheet identifier from link?


In [ ]:
var util = require('util');
var getInfo = importer.import('get google sheet info');
var getDataSheet = importer.import('google sheet array objects');
var addRow = importer.import('add row data google sheet');
var uuid = require('uuid/v1');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var project = 'spahaha-ea443';

function safeName(name) {
    return name.replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function getSheet(link) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name, title;
    
    return getInfo(link)
        // return assigned subdomain
        .then(info => {
            name = safeName(info.properties.title.replace(/\s*(configuration|config)\s*/ig, '')) + '-' + uuid().substr(0, 5);
            title = info.properties.title;
            return getDataSheet(purchaseId, 'Purchases');
        })
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(match) {
                console.log(`Purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
                return Promise.resolve(match.domain || match.bucket);
            }
        
            console.log(`Adding product row for ${docId} ${name}`)
            return addRow(purchaseId, {
                timestamp: Date.now(),
                name: title,
                email: '',
                address: '',
                domain: '',
                bucket: name + '.sheet-to-web.com',
                project: project,
                sheet: docId
            })
                .then(() => name + '.sheet-to-web.com')
        })
}

module.exports = getSheet;

google sheet handler?


In [ ]:
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await getSheet(req.query['link'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


sheet marketing import?


In [ ]:
var getDataSheet = importer.import('google sheet array objects');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

function importSheet(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var properties = {};
    
    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            return getTemplates(docId)
        })
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => collectTemplateResources(key, page, properties, templates, domain))
        .then(resources => {
            console.log(resources);
            return resources;
        })
}

module.exports = importSheet;


sheet marketing import handler?


In [ ]:
var importer = require('../Core');
var importSheet = importer.import('sheet marketing import');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await importSheet(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


setup sheet backend?


In [ ]:
var getDataSheet = importer.import('google sheet array objects');
var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var project;

    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            project = match.project;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
}

module.exports = setupBackend;

sheet backend handler?


In [ ]:
var importer = require('../Core');
var setupBackend = importer.import('setup sheet backend');

function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return setupBackend(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}
    
module.exports.handler = handler;


contact us handler?

In [ ]:
var importer = require('../Core');
var addRow = importer.import('add row data google sheet');

var contactId = '16b1Z0sQkYhtMUfP7qMRL3vRBPWQsRbSlntURkMqWGX0';
var redirect = 'https://www.shopilluminati.com/contact';

function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    console.log(`Adding contact row for ${req.body['name']}`)
    return addRow(contactId, {
        timestamp: Date.now(),
        name: req.body['name'],
        email: req.body['email'],
        subject: req.body['subject'],
        message: req.body['message'],
        responded: 0,
    })
        .then(r => res.redirect(redirect))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

    
module.exports.handler = handler;


package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
